# Analyse de données financières : Détection d'anomalies dans un porteuille d'actifs

In [ ]:
import quandl
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn
from sklearn.ensemble import IsolationForest
from sklearn.neighbors.kde import KernelDensity

# instruction pour plotter dans notebook
%matplotlib inline

# connection à quandl via l'api key
# quandl.ApiConfig.api_key = " 3CipYF1Y3fzjvDgg7E2n"

In [ ]:
# Lecture des données
# df = pd.read_csv("all_stocks_5yr.csv", index_col= [0,2],usecols=[0,4,6],parse_dates=[0])
df = pd.read_csv("all_stocks_5yr.csv",usecols=[0,4,6], parse_dates=[0])
df.head()
df.Date = pd.to_datetime(df.Date)
df = df.loc[(df.Date<datetime.datetime(2017, 8, 11,0,0,0)),:]
df = df.loc[df.Date>datetime.datetime(2015, 8, 11,0,0,0),:]
#df =df.dropna(axis=0, how='all').dropna(axis=1, how='any').dropna(axis=0, how='any')
#df.loc[df.Name=='MMM',:]

In [ ]:
# df sans index devient un df avec index
# df_mi = df.set_index(['Name','Date'])

In [ ]:
# On récupère les donnée sur deux ans, les données principales sont des données sur cinq ans


In [ ]:
# on unstack le multi index pour en faire un 
# df_unstack = df_mi.unstack(level=0)
#df_unstack

## Rendements

In [ ]:
# fonction de calcul de rendement
# def rends(dff):
#    return dff/dff.diff(1) - 1


In [ ]:
# Rendements journaliers obtenue à partit d'un tableau dynamique
df_rendements = df.pivot('Date', 'Name', 'Close').pct_change() #.reset_index(level=[0]).set_index(['Date'])

In [ ]:
# fontion de calcul de moyenne mobile sur le rendement
def rends_moov(dff):
    return dff.rolling(window =40).mean()

In [ ]:
# Calculate the moving average sur les rendements
df_moving_rendements = df_rendements.apply(rends_moov)
df_moving_rendements.plot(figsize=(12,8))

## Volatilités

In [ ]:
#fonction de calcul des volatilités sur une période
def volts(dff):
    return dff.rolling(40).std() * np.sqrt(40) 

In [ ]:
# Define the minumum of periods to consider 
# min_periods = 75
df_moving_volts = df_rendements.apply(volts)
df_moving_volts.plot(figsize=(12,8))

# Detection d'anomalies

# Détection sur les rendements

In [ ]:
# df_2 est de dataframe qui contient les données dont on doit détecter les anomalies
# Applications de l'isolation forest sur les données de rendements ou de volatilités
clf = IsolationForest(n_estimators=100, max_samples='auto')

# fit de l'estimateur
clf.fit(df_moving_rendements.transpose().dropna(axis=0, how='all').dropna(axis=1, how='any'))

# the anomaly score of the input samples. the lower the more abnormal.
scores_pred = pd.DataFrame(clf.decision_function(df_moving_rendements.transpose().dropna(axis=0, how='all').dropna(axis=1, how='any')),index=df_moving_rendements.transpose().dropna(axis=0, how='all').dropna(axis=1, how='any').index.values)

predictions = pd.DataFrame(clf.predict(df_moving_rendements.transpose().dropna(axis=0, how='all').dropna(axis=1, how='any')), index=df_moving_rendements.transpose().dropna(axis=0, how='all').dropna(axis=1, how='any').index.values)

In [ ]:
# Les scores de tous les individus
scores_pred.plot(figsize=(24,8))

In [ ]:
# Les anomalies prédits ou détectées par le modèle
predictions[predictions==-1].dropna().transpose().columns

In [ ]:
# le score des anomalies
scores_pred.loc[predictions[predictions==-1].dropna().transpose().columns,:].transpose()

In [ ]:
# le score des anomalies
scores_pred.loc[predictions[predictions==-1].dropna().transpose().columns,:].plot(figsize=(24,8), title='Scores des anomalies de rendements')

# Détection sur les volatilités

In [ ]:
# df_2 est de dataframe qui contient les données dont on doit détecter les anomalies
# Applications de l'isolation forest sur les données de rendements ou de volatilités
clf = IsolationForest(n_estimators=100, max_samples='auto')

# fit de l'estimateur
clf.fit(df_moving_volts.transpose().dropna(axis=0, how='all').dropna(axis=1, how='any'))

# the anomaly score of the input samples. the lower the more abnormal.
scores_pred2 = pd.DataFrame(clf.decision_function(df_moving_volts.transpose().dropna(axis=0, how='all').dropna(axis=1, how='any')),index=df_moving_volts.transpose().dropna(axis=0, how='all').dropna(axis=1, how='any').index.values)

# 
predictions2 = pd.DataFrame(clf.predict(df_moving_volts.transpose().dropna(axis=0, how='all').dropna(axis=1, how='any')), index=df_moving_volts.transpose().dropna(axis=0, how='all').dropna(axis=1, how='any').index.values)

In [ ]:
# Les scores de tous les individus
scores_pred2.plot(figsize=(24,8))

In [ ]:
# Les anomalies prédits ou détectées par le modèle
predictions2[predictions2==-1].dropna().transpose().columns

In [ ]:
# le score des anomalies
scores_pred2.loc[predictions2[predictions2==-1].dropna().transpose().columns,:].transpose()

In [ ]:
# Graphique des scores des anomalies

In [ ]:
scores_pred2.loc[predictions2[predictions2==-1].dropna().transpose().columns,:].plot(figsize=(24,8))

In [ ]:
# comparaison anomalies obetues selon le rendements vs selon la volatilité afin de comparer quelle sont détectées dans les deux cas
predictions2[predictions2==-1].dropna().transpose().columns.sort_values()==predictions[predictions==-1].dropna().transpose().columns.sort_values()

# Détection sur la fonction de densité

In [ ]:
# fonction d'estimation des fonction de densités des rendements des actifs
def fdensite(X):
    X_plot = np.linspace(-5, 10, 504)[:, np.newaxis]
    return np.exp(KernelDensity(kernel='gaussian', bandwidth=0.75).fit(X.dropna().values.reshape(-1, 1)).score_samples(X_plot))

In [ ]:
df_densites = df_rendements.apply(fdensite)
df_densites.to_csv('df_densite.csv')
df_densites.A.plot()

In [ ]:
# df_2 est de dataframe qui contient les données dont on doit détecter les anomalies
# Applications de l'isolation forest sur les données de rendements ou de volatilités
clf = IsolationForest(n_estimators=100, max_samples='auto')

# fit de l'estimateur
clf.fit(df_densites.transpose())

# the anomaly score of the input samples. the lower the more abnormal.
scores_pred3 = pd.DataFrame(clf.decision_function(df_densites.transpose()),index=df_densites.transpose().index.values)

# 
predictions3 = pd.DataFrame(clf.predict(df_densites.transpose()), index=df_densites.transpose().index.values)

In [ ]:
# Les scores de tous les individus
scores_pred3.plot(figsize=(24,8), title='Scores des desités des actifs')

In [ ]:
# Les anomalies prédites ou détectées par le modèle
predictions3[predictions3==-1].dropna().transpose().columns

In [ ]:
# le score des anomalies
scores_pred3.loc[predictions3[predictions3==-1].dropna().transpose().columns,:].transpose()

In [ ]:
# le score des anomalies
scores_pred3.loc[predictions3[predictions3==-1].dropna().transpose().columns,:].plot(figsize=(24,8), title='Scores des anomalies de rendements')

# Nielson siegel svensson détection sur densité

In [ ]:
#a = df_rendements.A.reset_index().A.sort_values().reset_index().A.to_csv('nss_data.csv')
a= pd.read_csv("sols.csv", index_col=0)

In [ ]:
a.head()

In [ ]:
# df_2 est de dataframe qui contient les données dont on doit détecter les anomalies
# Applications de l'isolation forest sur les données de rendements ou de volatilités
clf = IsolationForest(n_estimators=100, max_samples='auto')

# fit de l'estimateur
clf.fit(a)

# the anomaly score of the input samples. the lower the more abnormal.
scores_pred4 = pd.DataFrame(clf.decision_function(a),index=a.index.values)

# 
predictions4 = pd.DataFrame(clf.predict(a), index=a.index.values)

In [ ]:
# Les scores de tous les individus
scores_pred3.plot(figsize=(24,8), title='Scores des densité des actifs ')

In [ ]:
# Les anomalies prédites ou détectées par le modèle
predictions4[predictions4==-1].dropna().transpose().columns

In [ ]:
predictions3[predictions3==-1].dropna().transpose().columns